In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble  import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import random

from sklearn import metrics
from collections import Counter
import argparse
from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import ParameterGrid
import lightgbm as lgb

In [2]:
# GOOGLE COLAB SETUP

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!pwd

/content


In [0]:
#2. Get the file
data_path     = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/data'
codes_path    = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/codes'


#3. Read file as panda dataframe
train         = pd.read_csv(f'{data_path}/train_cleaned_no_punkt.csv') 
test_labelled = pd.read_csv(f'{data_path}/test_labelled_cleaned_no_punkt.csv') 
test_unlabelled = pd.read_csv(f'{data_path}/test_unlabelled_cleaned_no_punkt.csv') 

In [5]:
#2. Get the file
data_path     = '../data'
codes_path    = '../codes'


#3. Read file as panda dataframe
train         = pd.read_csv(f'{data_path}/train_cleaned_no_punkt.csv') 
test_labelled = pd.read_csv(f'{data_path}/test_labelled_cleaned_no_punkt.csv') 
test_unlabelled = pd.read_csv(f'{data_path}/test_unlabelled_cleaned_no_punkt.csv')

FileNotFoundError: ignored

In [0]:
train['mal'] = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
train.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
train.comment_text.fillna("empty", inplace=True)
train = train.drop_duplicates(subset=['comment_text', 'mal'])

test_labelled['mal'] = test_labelled[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
test_labelled.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
test_labelled.comment_text.fillna("empty", inplace=True)
test_labelled = test_labelled.drop_duplicates(subset=['comment_text'])

test_unlabelled.comment_text.fillna("empty", inplace=True)
test_unlabelled = test_unlabelled.drop_duplicates(subset=['comment_text'])

In [0]:
rs=42
def strat_split(strat=False):
  global rs
  if strat:
      from sklearn.model_selection import train_test_split      
      X_train1, X_test1, y_train1, y_test1  = train_test_split(train.drop('mal', axis=1), train.mal, stratify=train.mal, test_size=0.29, random_state=rs )
      X_train2, X_test2, y_train2, y_test2  = train_test_split(test_labelled.drop('mal', axis=1), test_labelled.mal, stratify=test_labelled.mal, test_size=0.29, random_state=rs)
      X = np.concatenate((X_train1.comment_text, X_train2.comment_text))
      y = np.concatenate((y_train1, y_train2))

      X_test = np.concatenate((X_test1.comment_text, X_test2.comment_text))
      y_test = np.concatenate((y_test1, y_test2))
  else:
      X = np.concatenate((train.comment_text, test_labelled.comment_text))
      y = np.concatenate((train.mal, test_labelled.mal))
      from sklearn.utils import shuffle
      from sklearn.model_selection import train_test_split
      X = shuffle(X, random_state=rs)
      y = shuffle(y, random_state=rs)      
      X, X_test, y, y_test  = train_test_split(X, y, stratify=y, test_size=0.3, random_state=rs )
      
  return X, X_test, y, y_test    
X, X_test, y, y_test = strat_split(True)

In [0]:
# Ridge

In [0]:
kf  = StratifiedKFold(n_splits=5, random_state=rs)
auc = []
roc = []
c   = 0

word_vectorizer   = TfidfVectorizer(
    sublinear_tf  = True,
    strip_accents = 'unicode',
    analyzer      = 'word',
    token_pattern = r'\w{1,}',
    stop_words    = 'english',
    ngram_range   = (1, 1),
    max_features  = 40000)



In [0]:
#auc_pr = 0.093434375824395
#auc_roc = 0.02343524395
c = 0
C_parameter = np.arange(0.1, 1, 0.1) 

# use best C

C_parameter = [1]

In [12]:
for c_p in C_parameter:  
      for c, (train_index, val_index) in enumerate(kf.split(X, y)):

            X_train, X_val      = X[train_index], X[val_index]
            y_train, y_val      = y[train_index], y[val_index] 
            word_vectorizer.fit(X_train)
            train_word_features = word_vectorizer.transform(X_train)
            val_word_features   = word_vectorizer.transform(X_val)
            y_train             = y_train.astype('int')
            y_val               = y_val.astype('int')
            classifier          = LogisticRegression(C=c_p, solver='sag')
            classifier.fit(train_word_features, y_train)
            probs               = classifier.predict_proba(val_word_features)[:,1]
            auc_roc             = roc_auc_score(y_val, probs)
            auc_pr              = average_precision_score(y_val, probs)

            if len(C_parameter)==1:
                # print performance
                print(f'---------------------------------------------')
                print(f'FOLD {c}: AUC PR-C = {round(auc_pr, 4)}, AUC ROC = {round(auc_roc, 4)}')
                print(f'---------------------------------------------')
                print(f'')

            auc.append(auc_pr)
            roc.append(auc_roc)
            c += 1

      if len(C_parameter)!=1:
             print(f'PARAMETER C = {c_p}')

      # print performance
      print(f'-----------------------------------------------')
      print(f'CV average: AUC PR-C = {round(np.array(auc).mean(), 4)}, AUC ROC = {round(np.array(roc).mean(), 4)}')
      print(f'-----------------------------------------------')
      print(f'')

---------------------------------------------
FOLD 0: AUC PR-C = 0.869, AUC ROC = 0.9689
---------------------------------------------

---------------------------------------------
FOLD 1: AUC PR-C = 0.8656, AUC ROC = 0.968
---------------------------------------------

---------------------------------------------
FOLD 2: AUC PR-C = 0.8671, AUC ROC = 0.9681
---------------------------------------------

---------------------------------------------
FOLD 3: AUC PR-C = 0.8143, AUC ROC = 0.9647
---------------------------------------------

---------------------------------------------
FOLD 4: AUC PR-C = 0.7758, AUC ROC = 0.9629
---------------------------------------------

-----------------------------------------------
CV average: AUC PR-C = 0.8383, AUC ROC = 0.9665
-----------------------------------------------



In [13]:
# TRAIN ON WHOLE DAATA AND PREDICT ON TEST
word_vectorizer.fit(X)
train_word_features  = word_vectorizer.transform(X)
test_word_features   = word_vectorizer.transform(X_test)
classifier           = LogisticRegression(C=1, solver='sag')
classifier.fit(train_word_features, y)
probs                = classifier.predict_proba(test_word_features)[:,1]
auc_roc              = roc_auc_score(y_test, probs)
auc_pr               = average_precision_score(y_test, probs)

# print performance
print(f'-----------------------------------------')
print(f'TEST: AUC PR-C = {round(auc_pr, 4)}, AUC ROC = {round(auc_roc, 4)}')
print(f'-----------------------------------------')
print(f'')

-----------------------------------------
TEST: AUC PR-C = 0.8328, AUC ROC = 0.967
-----------------------------------------



In [0]:
# RF

In [0]:
kf  = StratifiedKFold(n_splits=5, random_state=rs)
auc = []
roc = []
c   = 0

word_vectorizer   = TfidfVectorizer(
    sublinear_tf  = True,
    strip_accents = 'unicode',
    analyzer      = 'word',
    token_pattern = r'\w{1,}',
    stop_words    = 'english',
    ngram_range   = (1, 1),
    max_features  = 40000)

In [36]:
for c, (train_index, val_index) in enumerate(kf.split(X, y)):

            X_train, X_val      = X[train_index], X[val_index]
            y_train, y_val      = y[train_index], y[val_index] 
            word_vectorizer.fit(X_train)
            train_word_features = word_vectorizer.transform(X_train)
            val_word_features   = word_vectorizer.transform(X_val)
            y_train             = y_train.astype('int')
            y_val               = y_val.astype('int')
            classifier          = RandomForestClassifier(n_estimators=600, max_depth=None, max_features='auto', 
                                    min_samples_split=2, verbose = True, n_jobs=100)
            classifier.fit(train_word_features, y_train)
            probs               = classifier.predict_proba(val_word_features)[:,1]
            auc_roc             = roc_auc_score(y_val, probs)
            auc_pr              = average_precision_score(y_val, probs)

            
            # print performance
            print(f'---------------------------------------------')
            print(f'FOLD {c}: AUC PR-C = {round(auc_pr, 3)}, AUC ROC = {round(auc_roc, 3)}')
            print(f'---------------------------------------------')
            print(f'')

            auc.append(auc_pr)
            roc.append(auc_roc)
            c += 1


# print performance
print(f'-----------------------------------------------')
print(f'CV average: AUC PR-C = {round(np.array(auc).mean(), 3)}, AUC ROC = {round(np.array(roc).mean(), 3)}')
print(f'-----------------------------------------------')
print(f'')

[Parallel(n_jobs=100)]: Using backend ThreadingBackend with 100 concurrent workers.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-36-3f0823b2bd66>", line 12, in <module>
    classifier.fit(train_word_features, y_train)
  File "/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py", line 330, in fit
    for i, t in enumerate(trees))
  File "/usr/local/lib/python3.6/dist-packages/joblib/parallel.py", line 934, in __call__
    self.retrieve()
  File "/usr/local/lib/python3.6/dist-packages/joblib/parallel.py", line 833, in retrieve
    self._output.extend(job.get(timeout=self.timeout))
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 638, in get
    self.wait(timeout)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 635, in wait
    self._event.wait(timeout)
  File "/usr/lib/python3.6/threading.py", line 551, in wait
    signaled = self._cond.wait(timeout)
  File "/usr/lib/p

KeyboardInterrupt: ignored

In [0]:
# TRAIN ON WHOLE DAATA AND PREDICT ON TEST
word_vectorizer.fit(X)
train_word_features  = word_vectorizer.transform(X)
test_word_features   = word_vectorizer.transform(X_test)
classifier           = RandomForestClassifier(n_estimators=600, max_depth=None, max_features='auto', 
                                    min_samples_split=2, verbose = True, n_jobs=100)
classifier.fit(train_word_features, y)
probs                = classifier.predict_proba(test_word_features)[:,1]
auc_roc              = roc_auc_score(y_test, probs)
auc_pr               = average_precision_score(y_test, probs)

# print performance
print(f'-----------------------------------------')
print(f'TEST: AUC PR-C = {round(auc_pr, 3)}, AUC ROC = {round(auc_roc, 3)}')
print(f'-----------------------------------------')
print(f'')

In [0]:
# SVM

In [26]:
penalty = ['l2', 'l1']
alpha = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1]
max_iter = [1000, 10000, 15000]

# best parameters
penalty = ['l1']
alpha = [0.00001]
max_iter = [1000] 

for p in penalty:
  for a in alpha:
    for i in max_iter:
      auc = []
      roc = []
      c = 0
      # print performance
      print(f'-------------')
      print(f'penalty {p}')
      print(f'alpha {a}')
      print(f'max_iter {i}')
      print(f'-------------')
      
      for c, (train_index, val_index) in enumerate(kf.split(X, y)):

            X_train, X_val      = X[train_index], X[val_index]
            y_train, y_val      = y[train_index], y[val_index] 
            word_vectorizer.fit(X_train)
            train_word_features = word_vectorizer.transform(X_train)
            val_word_features   = word_vectorizer.transform(X_val)
            y_train             = y_train.astype('int')
            y_val               = y_val.astype('int')
            classifier          = SGDClassifier(n_jobs=20, random_state=rs, loss='log', shuffle=False, 
                                    penalty=p, alpha=a, max_iter=i)
            classifier.fit(train_word_features, y_train)
            probs               = classifier.predict_proba(val_word_features)[:,1]
            auc_roc             = roc_auc_score(y_val, probs)
            auc_pr              = average_precision_score(y_val, probs)
            
            
            print(f'---------------------------------------------')
            print(f'FOLD {c}: AUC PR-C = {round(auc_pr, 3)}, AUC ROC = {round(auc_roc, 3)}')
            print(f'---------------------------------------------')
            print(f'')

            auc.append(auc_pr)
            roc.append(auc_roc)
            c += 1


# print performance
print(f'-----------------------------------------------')
print(f'CV average: AUC PR-C = {round(np.array(auc).mean(), 3)}, AUC ROC = {round(np.array(roc).mean(), 3)}')
print(f'-----------------------------------------------')
print(f'')

-------------
penalty l1
alpha 1e-05
max_iter 1000
-------------
---------------------------------------------
FOLD 0: AUC PR-C = 0.862, AUC ROC = 0.964
---------------------------------------------

---------------------------------------------
FOLD 1: AUC PR-C = 0.862, AUC ROC = 0.962
---------------------------------------------

---------------------------------------------
FOLD 2: AUC PR-C = 0.864, AUC ROC = 0.963
---------------------------------------------

---------------------------------------------
FOLD 3: AUC PR-C = 0.812, AUC ROC = 0.963
---------------------------------------------

---------------------------------------------
FOLD 4: AUC PR-C = 0.781, AUC ROC = 0.963
---------------------------------------------

-----------------------------------------------
CV average: AUC PR-C = 0.836, AUC ROC = 0.963
-----------------------------------------------



In [27]:
# TRAIN ON WHOLE DAATA AND PREDICT ON TEST
word_vectorizer.fit(X)
train_word_features  = word_vectorizer.transform(X)
test_word_features   = word_vectorizer.transform(X_test)
classifier           = SGDClassifier(n_jobs=20, random_state=rs, loss='log', shuffle=False, 
                                    penalty=penalty[0], alpha=alpha[0], max_iter=max_iter[0])
classifier.fit(train_word_features, y)
probs                = classifier.predict_proba(test_word_features)[:,1]
auc_roc              = roc_auc_score(y_test, probs)
auc_pr               = average_precision_score(y_test, probs)

# print performance
print(f'-----------------------------------------')
print(f'TEST: AUC PR-C = {round(auc_pr, 3)}, AUC ROC = {round(auc_roc, 3)}')
print(f'-----------------------------------------')
print(f'')

-----------------------------------------
TEST: AUC PR-C = 0.827, AUC ROC = 0.9618
-----------------------------------------



In [0]:
# lightgbm

In [0]:
# muner of rounds
max_rounds = 600
stopping   = 600
verbose    = 200

# LGB parameters
lgb_params = {
    'boosting_type':     'gbdt',
    'objective':         'binary',
    'metrics':           'binary_logloss',
    'bagging_fraction':  0.9,
    'feature_fraction':  0.8,
    'lambda_l1':         0.1,
    'lambda_l2':         0.1,
    'min_split_gain':    0.01,
    'min_child_weight':  2,
    'min_child_samples': 20,
    'silent':            True,
    'verbosity':         100,
    'learning_rate':     0.1,
    'max_depth':         7,
    'num_leaves':        70,
    'scale_pos_weight':  1,
    'n_estimators':      max_rounds,
    'nthread' :          20,
    'random_state':      rs,
}

In [30]:
for c, (train_index, val_index) in enumerate(kf.split(X, y)):

            X_train, X_val      = X[train_index], X[val_index]
            y_train, y_val      = y[train_index], y[val_index] 
            word_vectorizer.fit(X_train)
            train_word_features = word_vectorizer.transform(X_train)
            val_word_features   = word_vectorizer.transform(X_val)
            y_train             = y_train.astype('int')
            y_val               = y_val.astype('int')
            classifier          = lgb.LGBMClassifier(**lgb_params) 
            classifier.fit(train_word_features, y_train)
            probs               = classifier.predict_proba(val_word_features)[:,1]
            auc_roc             = roc_auc_score(y_val, probs)
            auc_pr              = average_precision_score(y_val, probs)

            
            # print performance
            print(f'---------------------------------------------')
            print(f'FOLD {c}: AUC PR-C = {round(auc_pr, 3)}, AUC ROC = {round(auc_roc, 3)}')
            print(f'---------------------------------------------')
            print(f'')

            auc.append(auc_pr)
            roc.append(auc_roc)
            c += 1

# print performance
print(f'-----------------------------------------------')
print(f'CV average: AUC PR-C = {round(np.array(auc).mean(), 3)}, AUC ROC = {round(np.array(roc).mean(), 3)}')
print(f'-----------------------------------------------')
print(f'')

---------------------------------------------
FOLD 0: AUC PR-C = 0.851, AUC ROC = 0.96
---------------------------------------------

---------------------------------------------
FOLD 1: AUC PR-C = 0.854, AUC ROC = 0.959
---------------------------------------------

---------------------------------------------
FOLD 2: AUC PR-C = 0.85, AUC ROC = 0.958
---------------------------------------------

---------------------------------------------
FOLD 3: AUC PR-C = 0.798, AUC ROC = 0.956
---------------------------------------------

---------------------------------------------
FOLD 4: AUC PR-C = 0.764, AUC ROC = 0.958
---------------------------------------------

-----------------------------------------------
CV average: AUC PR-C = 0.83, AUC ROC = 0.961
-----------------------------------------------



In [31]:
# TRAIN ON WHOLE DAATA AND PREDICT ON TEST
word_vectorizer.fit(X)
train_word_features  = word_vectorizer.transform(X)
test_word_features   = word_vectorizer.transform(X_test)
classifier           = lgb.LGBMClassifier(**lgb_params) 
classifier.fit(train_word_features, y)
probs                = classifier.predict_proba(test_word_features)[:,1]
auc_roc              = roc_auc_score(y_test, probs)
auc_pr               = average_precision_score(y_test, probs)

# print performance
print(f'-----------------------------------------')
print(f'TEST: AUC PR-C = {round(auc_pr, 3)}, AUC ROC = {round(auc_roc, 3)}')
print(f'-----------------------------------------')
print(f'')

-----------------------------------------
TEST: AUC PR-C = 0.8189, AUC ROC = 0.958
-----------------------------------------

